In [1]:
import random
import math
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(time.time()) #To get a "really" random generator 

In [3]:
class Cell (object) :
    
    next_ID = 0
    
    def __init__(self, state = "NORMAL", clone = 0, fitness = 1, mutation_rate = 0.001):
        self.state = state
        self.fitness = fitness
        self.clone = clone
        self.mutation_rate = mutation_rate
        self.ID = Cell.next_ID
        self.treatment_duration = 0
        Cell.next_ID += 1
        
    def __str__(self):
        return "Cell #{0} of state : {1} and clone type {2}".format(self.ID, self.state, self.clone)
    
    def mutate (self, new_clone_id):
        if self.state == "NORMAL":
            self.state = "CANCEROUS"
            
        self.clone = new_clone_id
        self.fitness = random.uniform(0, 1)
        self.mutation_rate = random.uniform(0, 0.01)
        
    def treatment (self, alpha):
        self.fitness = self.fitness * alpha
        self.treatment_duration += 1

In [4]:
class Tissue ( object ) :
    
    def __init__(self, omega, alpha, generation_time, clones_init):
        self.generation_time = generation_time
        self.omega = omega
        self.alpha = alpha
        self.detection = False
        self.pop = []
        self.clones_pop = copy.deepcopy(clones_init)    
    
    def initial_population(self, N = 1000, prop_cancer = 0.01):
        
        nb_cancer_cells = round(prop_cancer *N)
        
        for i in range(N):
            if (i < nb_cancer_cells) :
                state = "CANCEROUS"
                clone = random.choice(range(1, len(self.clones_pop.keys())))
                
            else :
                state = "NORMAL"
                clone = 0
            
            fitness = self.clones_pop[clone]["fitness"]
            mutation_rate = self.clones_pop[clone]["mu"]  
                
            new_cell = Cell(state, clone, fitness, mutation_rate)
            self.pop.append(new_cell)
            self.clones_pop[clone]["freq"] += 1 
            
    def stats(self):
        nb_cells = len(self.pop)
        nb_normal_cells = self.clones_pop[0]["freq"]
        nb_cancer_cells = nb_cells - nb_normal_cells
        
        if (nb_cancer_cells >= self.omega * nb_cells):
            self.detection = True
            
        return (nb_cells, nb_normal_cells, nb_cancer_cells)
            
    def get_treatment(self):
        if (self.detection == True): #if the cancer has been detected
            for cell in self.pop:
                if (cell.state == "CANCEROUS") and (cell.treatment_duration < self.generation_time):
                    cell.treatment(self.alpha)
    
    def reproduce (self):
        minimum = 1
        candidat = self.pop[0]
        proba_selection = random.uniform(0, 1)
        
        for cell in self.pop :
            #selection of an individual with a proba proportional to the fitness
            #i.e. minimize the différence between the proba and the fitness
            if (abs( cell.fitness - proba_selection) < minimum):
                minimum = abs( cell.fitness - proba_selection)
                candidat = cell
        
        new_cell = Cell(candidat.state, candidat.clone, candidat.fitness, candidat.mutation_rate)
        
        
        #Proba of mutation : give birth to another clone type
        proba_mutation = random.uniform(0, 1)
        if proba_mutation < new_cell.mutation_rate :
            new_clone_id=len(self.clones_pop.keys())
            new_cell.mutate(new_clone_id)
            self.clones_pop[new_cell.clone] = self.clones_pop.get(new_cell.clone, {"fitness" :new_cell.fitness, 
                                         "mu": new_cell.mutation_rate, 
                                        "freq" : 1})
        else:
            self.clones_pop[new_cell.clone]["freq"] += 1
        self.pop.append(new_cell)
    
    def get_apoptose(self):
        cell = random.choice(self.pop)
        self.pop.remove(cell)
        self.clones_pop[cell.clone]["freq"] -= 1 

In [5]:
nb_init_clones = 3
clones_init = {0: {"fitness" : 1,"mu": 0.001, "freq" : 0}}

for i in range(1,nb_init_clones+1):
    clones_init[i] = clones_init.get(i, {"fitness" :round(random.uniform(0,1), 4), 
                                         "mu": round(random.uniform(0, 0.002), 4), 
                                        "freq" : 0})

In [6]:
def run (nb_runs, omega, alpha, T, treatment = True, verbose = True, clones_init=default_clone_init):
    
    my_tissue = tissue.Tissue(omega, alpha, T, clones_init=clones_init)
    my_tissue.initial_population(N = 1000, prop_cancer = 0.1)  #initial population of 1000 individus
    
    if verbose:
        print("Initilisation : ")
        print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells {}".format(my_tissue.stats()[0], my_tissue.stats()[1], my_tissue.stats()[2]) ) 

    for i in range (nb_runs):
        my_tissue.reproduce()
        my_tissue.get_apoptose()
        
        if (treatment) :
            my_tissue.get_treatment()
        
        nb_cells, nb_normal_cells, nb_cancer_cells = my_tissue.stats()
        
    for key in list(my_tissue.clones_pop.keys()):
        if (key != 0 and my_tissue.clones_pop[key]["freq"] == 0):
            del my_tissue.clones_pop[key]
    
    if verbose :
        print ( "Statistics after", nb_runs, "runs : ")
        print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells : {}".format(nb_cells, nb_normal_cells, nb_cancer_cells, "\n" ) )
        print ( "\n" )
        #print(my_tissue.clones_pop)
    return (my_tissue.stats())

NameError: name 'default_clone_init' is not defined

The simulation with a while loop takes a lot of time (it ends very rarely), maybe this is due to my code. Besides, in order to have a simulation, I decided to replace the while loop by a for loop.

In [ ]:
#Without treatment :
simulation1 = simulation (nb_runs=10000, omega=0.25, alpha=0.5, T=25, treatment=False, verbose=True, clones_init=clones_init)

In [ ]:
#With treatment :
simulation2 = simulation (nb_runs=10000, omega=0.25, alpha=0.5, T=25, treatment=True, verbose=True, clones_init=clones_init)

In [ ]:
def shannon_diversity (N, n):
    p = n/N
    return (- p *  math.log(p, 2)) #napierian logarithm

In [ ]:
shannon_diversity (1000, simulation1[2]) #without treatment

In [ ]:
shannon_diversity (1000, simulation2[2]) #with treatment